In [ ]:
# !pip install sqlalchemy


In [12]:
import pandas as pd
from sqlalchemy import create_engine, inspect

In [ ]:
# !ln -s ../../stockdata .

In [36]:
# engine = create_engine('sqlite:///NQ_DB.db')
engine = create_engine('sqlite:///stockdata/ESNQ_DB.db')

In [23]:
from sqlalchemy import create_engine, inspect

# Setup your engine
# engine = create_engine('sqlite:///data/NQ_DB.db')

# Use inspector to get table names
inspector = inspect(engine)
table_names = inspector.get_table_names()

print("Tables in the database:")
for table_name in table_names:
    print(table_name)

Tables in the database:


In [38]:
files = !ls stockdata/NQ*1M.csv
for file in files:
    print(file)


stockdata/NQ_2024-06-02_2024-06-10_1M.csv
stockdata/NQ_2024-06-09_2024-06-14_1M.csv
stockdata/NQ_2024-06-16_2024-06-25_1M.csv
stockdata/NQ_2024-06-23_2024-06-28_1M.csv
stockdata/NQ_2024-06-30_2024-07-07_1M.csv
stockdata/NQ_2024-07-07_2024-07-12_1M.csv
stockdata/NQ_2024-07-18_2024-08-02_1M.csv
stockdata/NQ_2024-07-21_2024-07-30_1M.csv
stockdata/NQ_2024-08-04_2024-08-13_1M.csv
stockdata/NQ_2024-08-11_2024-08-16_1M.csv
stockdata/NQ_2024-08-25_2024-08-30_1M.csv


In [ ]:
df = pd.read_csv(files[2])
df

In [ ]:
# df.to_sql('NQ', engine, if_exists='append')

In [39]:
def sql_importer (data, table_name):
    # Use inspector to check if the table exists
    inspector = inspect(engine)
    if not inspector.has_table(table_name):
        data.to_sql(table_name, engine, if_exists='append')
        print (f'New table created for {table_name} with {str(len(data))} rows')
    else:
        max_date = pd.read_sql(f'SELECT MAX(datetime) FROM {table_name}', engine).values[0][0]
        print(max_date)
        df = data[data.datetime > max_date]
        df.to_sql(table_name, engine, if_exists='append')
        print(str(len(df)) + ' new rows imported to DB')

In [44]:
for file in files:
    df = pd.read_csv(file)
    sql_importer(df, 'ES5M')

New table created for ES5M with 8050 rows
2024-06-10 12:11:00
5672 new rows imported to DB
2024-06-14 15:59:00
9420 new rows imported to DB
2024-06-25 15:59:00
4140 new rows imported to DB
2024-06-28 15:59:00
6511 new rows imported to DB
2024-07-07 18:19:00
6819 new rows imported to DB
2024-07-12 15:59:00
15080 new rows imported to DB
2024-08-02 15:59:00
0 new rows imported to DB
2024-08-02 15:59:00
9476 new rows imported to DB
2024-08-13 12:55:00
4324 new rows imported to DB
2024-08-16 15:59:00
6899 new rows imported to DB


In [43]:
pd.read_sql('ES1M', engine)

,index,datetime,symbol,open,high,low,close,volume
0,0,2024-06-02 17:00:00,CME_MINI:NQ1!,18590.25,18598.50,18576.00,18595.25,876.0
1,1,2024-06-02 17:01:00,CME_MINI:NQ1!,18594.25,18597.00,18587.75,18587.75,350.0
2,2,2024-06-02 17:02:00,CME_MINI:NQ1!,18588.50,18588.50,18562.25,18564.50,594.0
3,3,2024-06-02 17:03:00,CME_MINI:NQ1!,18563.75,18567.50,18557.25,18560.00,573.0
4,4,2024-06-02 17:04:00,CME_MINI:NQ1!,18560.50,18564.50,18553.75,18553.75,209.0
...,...,...,...,...,...,...,...,...
76386,6894,2024-08-30 15:55:00,CME_MINI:NQ1!,19596.00,19596.00,19591.75,19594.00,120.0
76387,6895,2024-08-30 15:56:00,CME_MINI:NQ1!,19594.00,19597.25,19593.50,19596.25,100.0
76388,6896,2024-08-30 15:57:00,CME_MINI:NQ1!,19596.25,19597.75,19594.00,19594.75,111.0
76389,6897,2024-08-30 15:58:00,CME_MINI:NQ1!,19595.00,19596.00,19591.75,19594.00,83.0


In [31]:
df = pd.read_sql('SELECT * FROM NQ5M', con=engine)
df

,index,datetime,symbol,open,high,low,close,volume
0,0,2024-05-12 17:00:00,CME_MINI:NQ1!,18238.00,18244.50,18233.25,18241.75,711.0
1,1,2024-05-12 17:05:00,CME_MINI:NQ1!,18241.75,18241.75,18228.75,18230.75,505.0
2,2,2024-05-12 17:10:00,CME_MINI:NQ1!,18230.75,18234.75,18230.50,18234.00,221.0
3,3,2024-05-12 17:15:00,CME_MINI:NQ1!,18233.75,18239.00,18233.25,18239.00,192.0
4,4,2024-05-12 17:20:00,CME_MINI:NQ1!,18238.50,18241.00,18237.00,18241.00,155.0
...,...,...,...,...,...,...,...,...
21886,7011,2024-08-30 15:35:00,CME_MINI:NQ1!,19604.75,19605.00,19601.00,19602.00,221.0
21887,7012,2024-08-30 15:40:00,CME_MINI:NQ1!,19601.75,19603.25,19596.00,19599.75,451.0
21888,7013,2024-08-30 15:45:00,CME_MINI:NQ1!,19599.75,19600.25,19595.50,19596.00,370.0
21889,7014,2024-08-30 15:50:00,CME_MINI:NQ1!,19596.50,19600.75,19595.00,19596.00,414.0


In [32]:
df = pd.read_sql('SELECT datetime, open, close FROM NQ5M', con=engine)
df

,datetime,open,close
0,2024-05-12 17:00:00,18238.00,18241.75
1,2024-05-12 17:05:00,18241.75,18230.75
2,2024-05-12 17:10:00,18230.75,18234.00
3,2024-05-12 17:15:00,18233.75,18239.00
4,2024-05-12 17:20:00,18238.50,18241.00
...,...,...,...
21886,2024-08-30 15:35:00,19604.75,19602.00
21887,2024-08-30 15:40:00,19601.75,19599.75
21888,2024-08-30 15:45:00,19599.75,19596.00
21889,2024-08-30 15:50:00,19596.50,19596.00


In [33]:
# df.describe().style.background_gradient(cmap='coolwarm')
df.describe()
df.describe().style.background_gradient(cmap='coolwarm')

,open,close
count,21891.000000,21891.000000
mean,19433.779898,19433.831026
std,725.540002,725.475703
min,17426.000000,17425.250000
25%,18778.750000,18779.000000
50%,19509.000000,19509.250000
75%,19974.250000,19974.375000
max,20972.250000,20971.750000


In [19]:
import pandas_ta as ta
macd = df.ta.macd(fast=13, slow=26)
macd

,MACD_13_26_9,MACDh_13_26_9,MACDs_13_26_9
0,NaN,NaN,NaN
1,NaN,NaN,NaN
2,NaN,NaN,NaN
3,NaN,NaN,NaN
4,NaN,NaN,NaN
...,...,...,...
17769,-34.766168,-4.404535,-30.361633
17770,-33.484629,-2.498397,-30.986232
17771,-32.199178,-0.970357,-31.228821
17772,-30.752269,0.381242,-31.133511


In [20]:
rsi = df.ta.rsi()
rsi

0              NaN
1              NaN
2              NaN
3              NaN
4              NaN
           ...    
17769    28.256008
17770    30.641413
17771    30.355382
17772    31.048540
17773    31.196034
Name: RSI_14, Length: 17774, dtype: float64

In [46]:
from sqlalchemy import create_engine, MetaData

# Setup your engine
# engine = create_engine('your_database_url')
# engine = create_engine('sqlite:///TEST_DB.db')
engine = create_engine('sqlite:///stockdata/ESNQ_DB.db')

# Reflect the database schema to get all table names
metadata = MetaData()
metadata.reflect(bind=engine)

print("Tables in the database:")
for table_name in metadata.tables.keys():
    print(table_name)

Tables in the database:
ES1M
ES5M
NQ1M
NQ5M
